In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
import datetime as dt
import matplotlib.pyplot as plt

In [53]:
!pwd

/home/tomi/code/Harlequinht


In [41]:
#Read CSV of raw flats data to ensure imputing doesn't affect correlation
df = pd.read_csv('combined_data/combined_flat2_csv.csv')

/tmp/ipykernel_1228/1565615824.py:2: DtypeWarning: Columns (22,53,62,71,80,89,98,107,116) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('combined_data/combined_flat2_csv.csv')


In [42]:
#ensuring win/loss is still within data as this is our y
df['win_or_lose']

0         0.0
1         0.0
2         0.0
3         1.0
4         0.0
         ... 
282001    0.0
282002    0.0
282003    0.0
282004    0.0
282005    0.0
Name: win_or_lose, Length: 282006, dtype: float64

In [48]:
#work out correlation between the features in the data set and y
win_lose_all_data = np.abs(df.corr(method = 'pearson'))
df_win_loss = win_lose[['win_or_lose']] #correlation between y and other vaiables

In [44]:
win_lose.stack().reset_index() #correlation between all features to check for self-correlation

,level_0,level_1,0
0,meeting_id,meeting_id,1.000000
1,meeting_id,event_number,0.053186
2,meeting_id,distance,0.020122
3,meeting_id,distance_raw_furlongs,0.020183
4,meeting_id,race_class,0.027277
...,...,...,...
3359,run_8_dsr,run_7_dsr,0.932725
3360,run_8_dsr,run_8_raw_post_race_rating_int,0.189099
3361,run_8_dsr,run_8_final_rating_int,0.174833
3362,run_8_dsr,run_8_win_lose,0.069755


In [50]:
df_win_loss.sort_values(by='win_or_lose', ascending=False) #correlation for all values against win/loss to see what is affecting it

,win_or_lose
win_or_lose,1.000000
finish_position,0.459894
post_race_master_rating_int,0.315602
starting_price,0.154336
run_1_raw_post_race_rating_int,0.134732
bet365_odds,0.127429
run_1_final_rating_int,0.112617
betfair_starting_price,0.110876
number,0.100044
run_1_win_lose,0.092000


In [51]:
# Converting the correlation matrix into a DataFrame for all data
corr_df = win_lose_all_data.stack().reset_index()

# Renaming the columns
corr_df.columns = ['feature_1', 'feature_2', 'correlation']

# Remove "self correlations"
no_self_correlation = (corr_df['feature_1'] != corr_df['feature_2'])
corr_df = corr_df[no_self_correlation]

In [52]:
corr_df.sort_values(by='correlation', ascending=False).head(20)

,feature_1,feature_2,correlation
176,distance_raw_furlongs,distance,0.999974
119,distance,distance_raw_furlongs,0.999974
1697,run_1_dsr,dslr,0.999602
899,dslr,run_1_dsr,0.999602
1592,run_1_final_rating_int,run_1_raw_post_race_rating_int,0.952085
1535,run_1_raw_post_race_rating_int,run_1_final_rating_int,0.952085
1828,run_2_final_rating_int,run_2_raw_post_race_rating_int,0.948956
1771,run_2_raw_post_race_rating_int,run_2_final_rating_int,0.948956
2064,run_3_final_rating_int,run_3_raw_post_race_rating_int,0.946594
2007,run_3_raw_post_race_rating_int,run_3_final_rating_int,0.946594


In [14]:
# # # new_df = pd.DataFrame() - self correlation

# df_temp_col_names = win_lose.columns.tolist()
# lst = []

# for i in range(len(win_lose)):
#     for j in range(len(win_lose.columns)):
#         if win_lose.iloc[i, j] != 0 and win_lose.iloc[i, j] != 1:
#             lst.append(f'{df_temp_col_names[i]} - {df_temp_col_names[j]} - {win_lose.iloc[i, j]}')

# lst

['Unnamed: 0 - meeting_id - 0.9506987836369463',
 'Unnamed: 0 - event_number - 0.03610220204736725',
 'Unnamed: 0 - distance - 0.02150568692720936',
 'Unnamed: 0 - distance_raw_furlongs - 0.02152113872469036',
 'Unnamed: 0 - race_class_normalised - 0.010392371951843261',
 'Unnamed: 0 - total_prize_money - 0.04543058443178062',
 'Unnamed: 0 - track_condition - 0.03843198908719944',
 'Unnamed: 0 - race_type - 0.015967083042205617',
 'Unnamed: 0 - track_type - 0.05461420408979158',
 'Unnamed: 0 - barrier - 0.030637374139352858',
 'Unnamed: 0 - number - 0.030463707887419512',
 'Unnamed: 0 - finish_position - 0.02536718239232971',
 'Unnamed: 0 - win_or_lose - 0.006554757784340701',
 'Unnamed: 0 - failed_to_finish_reason - 0.003642010728211993',
 'Unnamed: 0 - gear - 0.19078300723229036',
 'Unnamed: 0 - jockey_allowance - 0.016810210277771354',
 'Unnamed: 0 - handicap_weight - 0.1510328349610514',
 'Unnamed: 0 - jockey_id - 0.12475421780166049',
 'Unnamed: 0 - tainer_id - 0.17638526934950177